In [3]:
import pandas as pd
import numpy as np
# Load dataset
df = pd.read_csv("spotify_history.csv")

# Sanity checks
print("Dataset Shape:", df.shape)
print("\nFirst 5 Rows:")
print(df.head())

print("\nDataset Info:")
df.info()

print("\nBasic Statistics:")
print(df.describe(include="all"))
print("\nDuplicate rows before cleaning:", df.duplicated().sum())

# Remove duplicates
df = df.drop_duplicates()

print("Duplicate rows after cleaning:", df.duplicated().sum())
# Rename columns ONLY if they exist
rename_map = {
    "endTime": "end_time",
    "artistName": "artist_name",
    "trackName": "track_name",
    "msPlayed": "ms_played",
    "trackId": "track_id"
}

df = df.rename(columns={k: v for k, v in rename_map.items() if k in df.columns})

print("\nColumns after renaming:")
print(df.columns)
print("\nMissing values per column:")
print(df.isna().sum())
# Define critical columns safely
critical_cols = ["artist_name", "track_name", "end_time"]
critical_cols = [col for col in critical_cols if col in df.columns]

# Drop rows missing critical values
df = df.dropna(subset=critical_cols)

# Fill missing track IDs
if "track_id" in df.columns:
    df["track_id"] = df["track_id"].fillna("unknown")
# Convert end_time to datetime
if "end_time" in df.columns:
    df["end_time"] = pd.to_datetime(
        df["end_time"],
        errors="coerce",
        infer_datetime_format=True
    )

    # Drop invalid timestamps
    df = df.dropna(subset=["end_time"])

# Convert playback duration to numeric
if "ms_played" in df.columns:
    df["ms_played"] = pd.to_numeric(df["ms_played"], errors="coerce")
# Remove invalid or zero play durations
if "ms_played" in df.columns:
    df = df[df["ms_played"] > 0]

    # Convert to seconds & minutes
    df["seconds_played"] = df["ms_played"] / 1000
    df["minutes_played"] = df["seconds_played"] / 60
# Normalize artist names
if "artist_name" in df.columns:
    df["artist_name"] = df["artist_name"].str.lower().str.strip()

# Normalize track names
if "track_name" in df.columns:
    df["track_name"] = df["track_name"].str.lower().str.strip()

# Fix known inconsistencies
artist_map = {
    "weeknd": "the weeknd",
    "the weeknd ": "the weeknd",
    "taylor swift ": "taylor swift"
}

if "artist_name" in df.columns:
    df["artist_name"] = df["artist_name"].replace(artist_map)
print("\nFinal Dataset Shape:", df.shape)

print("\nFinal Missing Values:")
print(df.isna().sum())

# Ensure no duplicates remain
assert df.duplicated().sum() == 0, "Duplicate rows still exist!"

print("\nFinal Dataset Info:")
df.info()
df.to_csv("cleaned_spotify_streaming_history.csv", index=False)

print("\n✅ Spotify Streaming History cleaned successfully!")
print("📁 Saved as: cleaned_spotify_streaming_history.csv")


Dataset Shape: (149860, 11)

First 5 Rows:
        spotify_track_uri                   ts    platform  ms_played  \
0  2J3n32GeLmMjwuAzyhcSNe  2013-07-08 02:44:34  web player       3185   
1  1oHxIPqJyvAYHy0PVrDU98  2013-07-08 02:45:37  web player      61865   
2  487OPlneJNni3NWC8SYqhW  2013-07-08 02:50:24  web player     285386   
3  5IyblF777jLZj1vGHG2UD3  2013-07-08 02:52:40  web player     134022   
4  0GgAAB0ZMllFhbNc3mAodO  2013-07-08 03:17:52  web player          0   

                                      track_name        artist_name  \
0                            Say It, Just Say It       The Mowgli's   
1  Drinking from the Bottle (feat. Tinie Tempah)      Calvin Harris   
2                                    Born To Die       Lana Del Rey   
3                               Off To The Races       Lana Del Rey   
4                                      Half Mast  Empire Of The Sun   

                           album_name reason_start reason_end  shuffle  \
0                